In [2]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from neurora.stuff import limtozero
from neurora.rsa_plot import plot_brainrsa_rlts
from neurora.stuff import get_HOcort, mask_to
import nibabel as nib
from nilearn import plotting, image
from nilearn.image import smooth_img, threshold_img
import scipy.io as sio
from scipy.ndimage import gaussian_filter
from itertools import product
from scipy.stats import wilcoxon, ttest_1samp
from tqdm.auto import tqdm
from scipy.ndimage import label
import matplotlib.pyplot as plt
from nimare.utils import mni2tal, mm2vox, vox2mm
import pandas as pd
import pingouin as pg

plt.rcParams['figure.dpi'] = 300
warnings.filterwarnings("ignore", category=RuntimeWarning)

### Calculate RDMs

In [ ]:
def fmriRDM(fmri_data, ksize=[3, 3, 3], strides=[1, 1, 1], sub_opt=1, method="correlation", abs=False):

    """
    Calculate the Representational Dissimilarity Matrices (RDMs) based on fMRI data (searchlight)

    Parameters
    ----------
    fmri_data : array
        The fmri data.
        The shape of fmri_data must be [n_cons, n_subs, nx, ny, nz]. n_cons, nx, ny, nz represent the number of
        conditions, the number of subs & the size of fMRI-img, respectively.
    ksize : array or list [kx, ky, kz]. Default is [3, 3, 3].
        The size of the calculation unit for searchlight.
        kx, ky, kz represent the number of voxels along the x, y, z axis.
        kx, ky, kz should be odd.
    strides : array or list [sx, sy, sz]. Default is [1, 1, 1].
        The strides for calculating along the x, y, z axis.
    sub_opt: int 0 or 1. Default is 1.
        Return the subject-result or average-result.
        If sub_opt=0, return the average result.
        If sub_opt=1, return the results of each subject.
    method : string 'correlation' or 'euclidean'. Default is 'correlation'.
        The method to calculate the dissimilarities.
        If method='correlation', the dissimilarity is calculated by Pearson Correlation.
        If method='euclidean', the dissimilarity is calculated by Euclidean Distance, the results will be normalized.
    abs : boolean True or False. Default is True.
        Calculate the absolute value of Pearson r or not.

    Returns
    -------
    RDM : array
        The fMRI-Searchlight RDM.
        If sub_opt=0, the shape of RDMs is [n_x, n_y, n_z, n_cons, n_cons].
        If sub_opt=1, the shape of RDMs is [n_subs, n_x, n_y, n_cons, n_cons]
        n_subs, n_x, n_y, n_z represent the number of subjects & the number of calculation units for searchlight along
        the x, y, z axis.
    """

    if len(np.shape(fmri_data)) != 5:

        print("\nThe shape of input for fmriRDM() function must be [n_cons, n_subs, nx, ny, nz].\n")

        return "Invalid input!"

    # get the number of conditions, subjects and the size of the fMRI-img
    cons, subs, nx, ny, nz = np.shape(fmri_data)

    # the size of the calculation units for searchlight
    kx = ksize[0]
    ky = ksize[1]
    kz = ksize[2]

    # strides for calculating along the x, y, z axis
    sx = strides[0]
    sy = strides[1]
    sz = strides[2]

    # calculate the number of the calculation units in the x, y, z directions
    n_x = int((nx - kx) / sx)+1
    n_y = int((ny - ky) / sy)+1
    n_z = int((nz - kz) / sz)+1

    # initialize the data for calculating the RDM
    data = np.full([n_x, n_y, n_z, cons, kx*ky*kz, subs], np.nan)

    print("\nComputing RDMs")
    
    print("Assign the voxels...")
    # assignment
    for x in tqdm(range(n_x)):
        for y in range(n_y):
            for z in range(n_z):
                for i in range(cons):
                    
                    block = fmri_data[i, :, x*sx:x*sx+kx, y*sy:y*sy+ky, z*sz:z*sz+kz].reshape(subs, -1)
                    data[x, y, z, i, :, :] = block.T

    print("Done")

    # shape of data: [n_x, n_y, n_z, cons, kx*ky*kz, subs]
    #              ->[subs, n_x, n_y, n_z, cons, kx*ky*kz]
    data = np.transpose(data, (5, 0, 1, 2, 3, 4))

    # flatten the data for different calculating conditions
    data = np.reshape(data, [subs, n_x, n_y, n_z, cons, kx*ky*kz])

    # initialize the RDMs
    subrdms = np.full([subs, 2, n_x, n_y, n_z, cons, cons], np.nan)
    
    for sub in range(subs):
        
        print("Sub"+str(sub+1)+": Searchlight...")
        
        for x in tqdm(range(n_x)):
            for y in range(n_y):
                for z in range(n_z):
                    
                    voxel_data = data[sub, x, y, z]
                    
                    if not np.isnan(voxel_data).any():
                        voxel_data_means = np.mean(voxel_data, axis=1)  # shape = (cons,)
                        subrdms[sub, 0, x, y, z] = np.abs(voxel_data_means[:, None] - voxel_data_means[None, :])
                        subrdms[sub, 1, x, y, z] = 1 - np.corrcoef((voxel_data-np.average(voxel_data))/np.std(voxel_data))
    
        print("Done.")

    if sub_opt == 0:
        
        # average the RDMs
        rdms = np.average(subrdms, axis=0)

        return rdms

    if sub_opt == 1:

        return subrdms

    print("\nRDMs computing finished!")

In [ ]:
subs = ['1', '2', '3', '4', '5', '7', '8', '9', '10']

for sub in subs:
    print('Sub: '+sub)
    data = np.array(sio.loadmat('sub'+sub+'/betaMatrix.mat')['betamatrix'])
    data = np.transpose(data, (3, 0, 1, 2))
    data = np.reshape(data, [64, 1, 87, 72, 69])
    fmri_rdms = fmriRDM(data)
    np.save('fMRI_RDMs/sub'+sub+'_fmri_rdms.npy', fmri_rdms[0])

### Searchlight RSA

In [ ]:
model_rdms = np.load('../model_rdms.npy')[[0, 1, 2, 3, 4, 5, 6, 9, 10]]

triu_indices = np.triu_indices(64, k=1)

model_vs = model_rdms[:, triu_indices[0], triu_indices[1]]

sub_ids = ['1', '2', '3', '4', '5', '7', '8', '9', '10']

for sub_id in sub_ids:
    
    fmri_rdms = np.load('fMRI_RDMs/sub'+sub+'_fmri_rdms.npy')[1]
    n_x, n_y, n_z = fmri_rdms.shape[:3]

    corrs = np.zeros([9, n_x, n_y, n_z, 2])

    for x in tqdm(range(n_x)):
        for y in range(n_y):
            for z in range(n_z):
            
                fmri_rdm = fmri_rdms[x, y, z]
                fmri_v = fmri_rdm[triu_indices[0], triu_indices[1]]
            
                if not np.isnan(fmri_v).any():
            
                    for model_index in range(9):
                        data = {
                            'y': fmri_v,
                            'x': model_vs[model_index],
                        }
                        covars = {f'x{covar_index}': model_vs[covar_index] for covar_index in range(9) if covar_index != model_index}
                        data.update(covars)
                        df = pd.DataFrame(data)
                        stats = pg.partial_corr(data=df, x='x', y='y', covar=list(covars.keys()),
                                            alternative='greater', method='spearman')
                        corrs[model_index, x, y, z, 0] = stats['r'][0]
                        corrs[model_index, x, y, z, 1] = stats['p-val'][0]
                
    np.save('sub'+sub_id+'/sub'+sub_id+'_partialcorrs_9.npy', corrs)
    
    fmri_rdms = np.load('fMRI_RDMs/sub'+sub+'_fmri_rdms.npy')[0]
    n_x, n_y, n_z = fmri_rdms.shape[:3]

    corrs = np.zeros([9, n_x, n_y, n_z, 2])

    for x in tqdm(range(n_x)):
        for y in range(n_y):
            for z in range(n_z):
            
                fmri_rdm = fmri_rdms[x, y, z]
                fmri_v = fmri_rdm[triu_indices[0], triu_indices[1]]
            
                if not np.isnan(fmri_v).any():
            
                    for model_index in range(9):
                        data = {
                            'y': fmri_v,
                            'x': model_vs[model_index],
                        }
                        covars = {f'x{covar_index}': model_vs[covar_index] for covar_index in range(9) if covar_index != model_index}
                        data.update(covars)
                        df = pd.DataFrame(data)
                        stats = pg.partial_corr(data=df, x='x', y='y', covar=list(covars.keys()),
                                            alternative='greater', method='spearman')
                        corrs[model_index, x, y, z, 0] = stats['r'][0]
                        corrs[model_index, x, y, z, 1] = stats['p-val'][0]
                
    np.save('sub'+sub_id+'/sub'+sub_id+'_partialcorrs_amp_9.npy', corrs)

Codes for ROI-based RSA and RSA for geometric distance representations should be similar to the above.

In [3]:
demo_data = nib.load('sub4/left32_vs_right32.nii')
affine = demo_data.affine
affine

array([[ 0.,  0., -2., 69.],
       [-2.,  0.,  0., 71.],
       [ 0., -2.,  0., 76.],
       [ 0.,  0.,  0.,  1.]])

In [4]:
def identify_clusters_3d(binary_matrix, r_matrix, voxel_threshold=25):
    
    cluster_labels, cluster_count = label(binary_matrix)

    filtered_cluster_masses = []
    valid_cluster_ids = []
    for cluster_id in range(1, cluster_count + 1):
        cluster_mask = (cluster_labels == cluster_id)
        cluster_size = np.sum(cluster_mask)

        if cluster_size >= voxel_threshold:
            cluster_mass = np.sum(r_matrix[cluster_mask])
            filtered_cluster_masses.append(cluster_mass)
            valid_cluster_ids.append(cluster_id)

    filtered_labels = np.zeros_like(cluster_labels)
    for new_id, valid_id in enumerate(valid_cluster_ids, start=1):
        filtered_labels[cluster_labels == valid_id] = new_id

    max_cluster_mass = max(filtered_cluster_masses) if filtered_cluster_masses else 0
    filtered_cluster_masses = np.array(filtered_cluster_masses)

    return filtered_labels, filtered_cluster_masses, len(valid_cluster_ids), max_cluster_mass


In [6]:
def corr_perm_save_nii(avgcorrs, thresholdcorrs, affine, maskfile=None, filename=None, size=[60, 60, 60],
                       ksize=[3, 3, 3], strides=[1, 1, 1], cluster_correction=False, nsubs=3):

    if len(np.shape(avgcorrs)) != 3 or len(np.shape(affine)) != 2 or np.shape(affine)[0] != 4 or np.shape(affine)[1] != 4:

        return "Invalid input!"

    # get the size of the fMRI-img
    nx = size[0]
    ny = size[1]
    nz = size[2]

    # the size of the calculation units for searchlight
    kx = ksize[0]
    ky = ksize[1]
    kz = ksize[2]

    rx = int((kx-1)/2)
    ry = int((ky-1)/2)
    rz = int((kz-1)/2)
    
    # calculate the number of the calculation units in the x, y, z directions
    n_x = np.shape(avgcorrs)[0]
    n_y = np.shape(avgcorrs)[1]
    n_z = np.shape(avgcorrs)[2]

    avgcorrs = np.arctanh(avgcorrs)
    thresholdcorrs = np.arctanh(thresholdcorrs)

    # initialize the img array to save the r-value for each voxel
    img_nii = np.zeros([nx, ny, nz], dtype=np.float64)
    
    maskdata = nib.load(maskfile)
    mask_nii = maskdata.get_fdata()
    mask_affine = maskdata.affine
    mask_affine_inv = np.linalg.inv(mask_affine)
    
    mask_cov = np.zeros([nx, ny, nz])
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                coord = np.dot(affine, np.array([i, j, k, 1]))
                x, y, z = coord[:3]
                mask_coords = np.dot(mask_affine_inv, np.array([x, y, z, 1]))
                mask_i, mask_j, mask_k = mask_coords[:3]
                mask_cov[i, j, k] = mask_nii[int(mask_i), int(mask_j), int(mask_k)]
    
    for i in range(n_x):
        for j in range(n_y):
            for k in range(n_z):
                if mask_cov[i+rx, j+ry, k+rz] == 0:
                    avgcorrs[i, j, k] = 0
                    
    avgcorrs[:rx] = 0
    avgcorrs[-rx:] = 0
    avgcorrs[:, :ry] = 0
    avgcorrs[:, -ry:] = 0
    avgcorrs[:, :, :rz] = 0
    avgcorrs[:, :, -rz:] = 0
    
    stats = np.zeros([n_x, n_y, n_z])
    for i in range(n_x):
        for j in range(n_y):
            for k in range(n_z):
                if avgcorrs[i, j, k] > thresholdcorrs[i, j, k]:
                    stats[i, j, k] = 1
    print(np.sum(stats>0))
    print('Permutation Done.')

    # record the valid voxels
    # [n_x, n_y, n_z] expanses into [nx, ny, nz] based on ksize & strides
    for i in range(n_x):
        for j in range(n_y):
            for k in range(n_z):
                
                if stats[i, j, k] == 1:
                    img_nii[i+rx, j+ry, k+rz] = avgcorrs[i, j, k]
    
    
    # set filename for result .nii file
    if filename == None:
        filename = "rsa_result.nii"
    else:
        q = ".nii" in filename

        if q == True:
            filename = filename
        else:
            filename = filename+".nii"

    #print(filename)

    #print("Save RSA results.")

    # save the .nii file for RSA results
    file = nib.Nifti1Image(img_nii, affine)

    # save the result
    nib.save(file, filename)


    # determine if it has results
    norlt = np.isnan(img_nii).all()
    if norlt == True:
        print("No RSA results.")

    print("File("+filename+") saves successfully!")

    return img_nii

In [7]:
def corr_perm_save_nii1(avgcorrs, stats, affine, maskfile=None, filename=None, size=[60, 60, 60],
                       ksize=[3, 3, 3], strides=[1, 1, 1], cluster_correction=False, nsubs=3):

    if len(np.shape(avgcorrs)) != 3 or len(np.shape(affine)) != 2 or np.shape(affine)[0] != 4 or np.shape(affine)[1] != 4:

        return "Invalid input!"

    # get the size of the fMRI-img
    nx = size[0]
    ny = size[1]
    nz = size[2]

    # the size of the calculation units for searchlight
    kx = ksize[0]
    ky = ksize[1]
    kz = ksize[2]

    rx = int((kx-1)/2)
    ry = int((ky-1)/2)
    rz = int((kz-1)/2)
    
    # calculate the number of the calculation units in the x, y, z directions
    n_x = np.shape(avgcorrs)[0]
    n_y = np.shape(avgcorrs)[1]
    n_z = np.shape(avgcorrs)[2]

    avgcorrs = np.arctanh(avgcorrs)

    # initialize the img array to save the r-value for each voxel
    img_nii = np.zeros([nx, ny, nz], dtype=np.float64)
    
    maskdata = nib.load(maskfile)
    mask_nii = maskdata.get_fdata()
    mask_affine = maskdata.affine
    mask_affine_inv = np.linalg.inv(mask_affine)
    
    mask_cov = np.zeros([nx, ny, nz])
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                coord = np.dot(affine, np.array([i, j, k, 1]))
                x, y, z = coord[:3]
                mask_coords = np.dot(mask_affine_inv, np.array([x, y, z, 1]))
                mask_i, mask_j, mask_k = mask_coords[:3]
                mask_cov[i, j, k] = mask_nii[int(mask_i), int(mask_j), int(mask_k)]
    
    for i in range(n_x):
        for j in range(n_y):
            for k in range(n_z):
                if mask_cov[i+rx, j+ry, k+rz] == 0:
                    avgcorrs[i, j, k] = 0
                    
    avgcorrs[:rx] = 0
    avgcorrs[-rx:] = 0
    avgcorrs[:, :ry] = 0
    avgcorrs[:, -ry:] = 0
    avgcorrs[:, :, :rz] = 0
    avgcorrs[:, :, -rz:] = 0
    
    print(np.sum(stats>0))
    print('Permutation Done.')

    # record the valid voxels
    # [n_x, n_y, n_z] expanses into [nx, ny, nz] based on ksize & strides
    for i in range(n_x):
        for j in range(n_y):
            for k in range(n_z):
                
                if stats[i, j, k] == 1:
                    img_nii[i+rx, j+ry, k+rz] = avgcorrs[i, j, k]
    
    
    # set filename for result .nii file
    if filename == None:
        filename = "rsa_result.nii"
    else:
        q = ".nii" in filename

        if q == True:
            filename = filename
        else:
            filename = filename+".nii"

    #print(filename)

    #print("Save RSA results.")

    # save the .nii file for RSA results
    file = nib.Nifti1Image(img_nii, affine)

    # save the result
    nib.save(file, filename)


    # determine if it has results
    norlt = np.isnan(img_nii).all()
    if norlt == True:
        print("No RSA results.")

    print("File("+filename+") saves successfully!")

    return img_nii

In [23]:
subs = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
corrs = np.zeros([9, 85, 70, 67, len(subs)])
i = 0
for sub in subs:
    sub_corrs = np.load('sub'+sub+'/partialcorrs_9.npy')[:, :, :, :, 0]
    corrs[:, :, :, :, i] = sub_corrs
    i += 1
avgcorrs = np.average(corrs, axis=4)
print(avgcorrs.shape)

thresholdcorrs = np.load('perm/perm_threshold_9_990.npy')

titles = ['X', 'Y', 'Z', 'r', 'theta', 'r-3D', 'phi',
          'r-3D-head-centered', 'phi-head-centered']
for i in range(9):
    corr_perm_save_nii(avgcorrs[i], thresholdcorrs[i], affine, maskfile='ch2better.nii.gz',
                       filename='group/'+titles[i]+'_pat.nii',
                       size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
    
avgcorrs_2D_cartesian_avg = (avgcorrs[0] + avgcorrs[1])/2
avgcorrs_2D_polar_avg = (avgcorrs[3] + avgcorrs[4])/2
avgcorrs_3D_cartesian_avg = (avgcorrs[0] + avgcorrs[1] + avgcorrs[2])/3
avgcorrs_3D_cylindrical_avg = (avgcorrs[3] + avgcorrs[4] + avgcorrs[2])/3
avgcorrs_3D_spherical_avg = (avgcorrs[5] + avgcorrs[4] + avgcorrs[6])/3
avgcorrs_3D_spherical_hc_avg = (avgcorrs[7] + avgcorrs[4] + avgcorrs[8])/3
thresholdcorrs_2D_cartesian_avg = (thresholdcorrs[0] + thresholdcorrs[1])/2
thresholdcorrs_2D_polar_avg = (thresholdcorrs[3] + thresholdcorrs[4])/2
thresholdcorrs_3D_cartesian_avg = (thresholdcorrs[0] + thresholdcorrs[1] + thresholdcorrs[2])/3
thresholdcorrs_3D_cylindrical_avg = (thresholdcorrs[3] + thresholdcorrs[4] + thresholdcorrs[2])/3
thresholdcorrs_3D_spherical_avg = (thresholdcorrs[5] + thresholdcorrs[4] + thresholdcorrs[6])/3
thresholdcorrs_3D_spherical_hc_avg = (thresholdcorrs[7] + thresholdcorrs[4] + thresholdcorrs[8])/3
corr_perm_save_nii(avgcorrs_2D_cartesian_avg, thresholdcorrs_2D_cartesian_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_cartesian_avg_pat.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_2D_polar_avg, thresholdcorrs_2D_polar_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_polar_avg_pat.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D_cartesian_avg, thresholdcorrs_3D_cartesian_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_cartesian_avg_pat.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D_cylindrical_avg, thresholdcorrs_3D_cylindrical_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_cylindrical_avg_pat.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D_spherical_avg, thresholdcorrs_3D_spherical_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_spherical_avg_pat.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D_spherical_hc_avg, thresholdcorrs_3D_spherical_hc_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_spherical_hc_avg_pat.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

coords = ['2D_cartesian', '2D_polar',
          '3D_cartesian', '3D_cylindrical', '3D_spherical', '3D_spherical_hc']
features = ['X', 'Y', 'Z', 'r', 'theta', 'r-3D', 'phi',
            'r-3D-head-centered', 'phi-head-centered']

data6 = np.zeros([6, 87, 72, 69])
for i in range(len(coords)):
    data6[i] = nib.load('group/'+coords[i]+'_avg_pat.nii').get_fdata()

data9 = np.zeros([9, 87, 72, 69])
for i in range(len(features)):
    data9[i] = nib.load('group/'+features[i]+'_pat.nii').get_fdata()

for i in range(2):
    data_argmax = np.zeros([87, 72, 69])
    for x in range(87):
        for y in range(72):
            for z in range(69):
                if data6[i, x, y, z] > 0 and np.argmax(data6[:2, x, y, z]) == i:
                    data_argmax[x, y, z] = data6[i, x, y, z]
    file = nib.Nifti1Image(data_argmax, affine)
    nib.save(file, 'group/coord_argmax_'+coords[i]+'_pat.nii')
    
for i in range(4):
    data_argmax = np.zeros([87, 72, 69])
    for x in range(87):
        for y in range(72):
            for z in range(69):
                if data6[i+2, x, y, z] > 0 and np.argmax(data6[2:, x, y, z]) == i:
                    data_argmax[x, y, z] = data6[i+2, x, y, z]
    file = nib.Nifti1Image(data_argmax, affine)
    nib.save(file, 'group/coord_argmax_'+coords[i+2]+'_pat.nii')

data_emergent = np.zeros([6, 87, 72, 69])
data_complete = np.zeros([6, 87, 72, 69])

for x in range(87):
    for y in range(72):
        for z in range(69):
            if data6[0, x, y, z] > 0:
                if data9[0, x, y, z] > 0 or data9[1, x, y, z] > 0:
                    data_emergent[0, x, y, z] = data6[0, x, y, z]
                if data9[0, x, y, z] > 0 and data9[1, x, y, z] > 0:
                    data_complete[0, x, y, z] = data6[0, x, y, z]
            if data6[1, x, y, z] > 0:
                if data9[3, x, y, z] > 0 or data9[4, x, y, z] > 0:
                    data_emergent[1, x, y, z] = data6[1, x, y, z]
                if data9[3, x, y, z] > 0 and data9[4, x, y, z] > 0:
                    data_complete[1, x, y, z] = data6[1, x, y, z]
            if data6[2, x, y, z] > 0:
                if data9[0, x, y, z] > 0 or data9[1, x, y, z] > 0:
                    data_emergent[2, x, y, z] = data6[2, x, y, z]
                if data9[0, x, y, z] > 0 and data9[1, x, y, z] > 0 and data9[2, x, y, z] > 0:
                    data_complete[2, x, y, z] = data6[2, x, y, z]
            if data6[3, x, y, z] > 0:
                if data9[3, x, y, z] > 0:
                    data_emergent[3, x, y, z] = data6[3, x, y, z]
                if data9[3, x, y, z] > 0 and data9[4, x, y, z] > 0 and data9[2, x, y, z] > 0:
                    data_complete[3, x, y, z] = data6[3, x, y, z]
            if data6[4, x, y, z] > 0:
                if data9[5, x, y, z] > 0 or data9[6, x, y, z] > 0:
                    data_emergent[4, x, y, z] = data6[4, x, y, z]
                if data9[5, x, y, z] > 0 and data9[4, x, y, z] > 0 and data9[6, x, y, z] > 0:
                    data_complete[4, x, y, z] = data6[4, x, y, z]
            if data6[5, x, y, z] > 0:
                if data9[7, x, y, z] > 0 or data9[8, x, y, z] > 0:
                    data_emergent[5, x, y, z] = data6[5, x, y, z]
                if data9[7, x, y, z] > 0 and data9[4, x, y, z] > 0 and data9[8, x, y, z] > 0:
                    data_complete[5, x, y, z] = data6[5, x, y, z]
for i in range(6):
    file = nib.Nifti1Image(data_emergent[i], affine)
    nib.save(file, 'group/coord_emergent_'+coords[i]+'_pat.nii')
    file = nib.Nifti1Image(data_complete[i], affine)
    nib.save(file, 'group/coord_complete_'+coords[i]+'_pat.nii')

max_3d_polar = np.zeros([85, 70, 67])

stats = np.zeros([85, 70, 67])
for x in range(85):
    for y in range(70):
        for z in range(67):
            if (avgcorrs_3D_cylindrical_avg[x, y, z]>thresholdcorrs_3D_cylindrical_avg[x, y, z]) or (avgcorrs_3D_spherical_avg[x, y, z]>thresholdcorrs_3D_spherical_avg[x, y, z]) or (avgcorrs_3D_spherical_hc_avg[x, y, z]>thresholdcorrs_3D_spherical_hc_avg[x, y, z]):
                max_3d_polar[x, y, z] = np.max([avgcorrs_3D_cylindrical_avg[x, y, z], avgcorrs_3D_spherical_avg[x, y, z], avgcorrs_3D_spherical_hc_avg[x, y, z]])
                stats[x, y, z] = 1  
corr_perm_save_nii1(max_3d_polar, stats, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_polar_avg_max_pat.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

cartesian_polar_2D = avgcorrs_2D_cartesian_avg - avgcorrs_2D_polar_avg
cartesian_polar_3D = avgcorrs_3D_cartesian_avg - max_3d_polar

stats_2D = np.zeros([85, 70, 67])
stats_3D = np.zeros([85, 70, 67])
for x in range(85):
    for y in range(70):
        for z in range(67):
            if (avgcorrs_2D_cartesian_avg[x, y, z] > thresholdcorrs_2D_cartesian_avg[x, y, z]) or (avgcorrs_2D_polar_avg[x, y, z] > thresholdcorrs_2D_polar_avg[x, y, z]):
                stats_2D[x, y, z] = 1
            if (avgcorrs_3D_cartesian_avg[x, y, z] > thresholdcorrs_3D_cartesian_avg[x, y, z]) or (avgcorrs_3D_cylindrical_avg[x, y, z]>thresholdcorrs_3D_cylindrical_avg[x, y, z]) or (avgcorrs_3D_spherical_avg[x, y, z]>thresholdcorrs_3D_spherical_avg[x, y, z]) or (avgcorrs_3D_spherical_hc_avg[x, y, z]>thresholdcorrs_3D_spherical_hc_avg[x, y, z]):
                stats_3D[x, y, z] = 1
corr_perm_save_nii1(cartesian_polar_2D, stats_2D, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_cartesian_polar_pat.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii1(cartesian_polar_3D, stats_3D, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_cartesian_polar_pat.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

avgcorrs_2D = (avgcorrs[0] + avgcorrs[1] + avgcorrs[3] + avgcorrs[4])/4
avgcorrs_3D = (avgcorrs[5] + avgcorrs[6] + avgcorrs[7] + avgcorrs[8])/4
thresholdcorrs_2D = (thresholdcorrs[0] + thresholdcorrs[1] + thresholdcorrs[3] + thresholdcorrs[4])/4
thresholdcorrs_3D = (thresholdcorrs[5] + thresholdcorrs[6] + thresholdcorrs[7] + thresholdcorrs[8])/4
corr_perm_save_nii(avgcorrs_2D, thresholdcorrs_2D, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_features_avg_pat.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D, thresholdcorrs_3D, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_features_avg_pat.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

stats_2D = np.zeros([85, 70, 67])
stats_3D = np.zeros([85, 70, 67])
for x in range(85):
    for y in range(70):
        for z in range(67):
            if (avgcorrs[0, x, y, z] > thresholdcorrs[0, x, y, z]) or (avgcorrs[1, x, y, z] > thresholdcorrs[1, x, y, z])or (avgcorrs[3, x, y, z] > thresholdcorrs[3, x, y, z]) or (avgcorrs[4, x, y, z] > thresholdcorrs[4, x, y, z]):
                stats_2D[x, y, z] = 1
            if (avgcorrs[5, x, y, z] > thresholdcorrs[5, x, y, z]) or (avgcorrs[6, x, y, z] > thresholdcorrs[6, x, y, z])or (avgcorrs[7, x, y, z] > thresholdcorrs[7, x, y, z]) or (avgcorrs[8, x, y, z] > thresholdcorrs[8, x, y, z]):
                stats_3D[x, y, z] = 1
corr_perm_save_nii1(avgcorrs_2D, stats_2D, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_features_avg_and_pat.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii1(avgcorrs_3D, stats_3D, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_features_avg_and_pat.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

(9, 85, 70, 67)
37106
Permutation Done.
File(group/X_pat.nii) saves successfully!
25396
Permutation Done.
File(group/Y_pat.nii) saves successfully!
3353
Permutation Done.
File(group/Z_pat.nii) saves successfully!
3845
Permutation Done.
File(group/r_pat.nii) saves successfully!
16959
Permutation Done.
File(group/theta_pat.nii) saves successfully!
13890
Permutation Done.
File(group/r-3D_pat.nii) saves successfully!
6703
Permutation Done.
File(group/phi_pat.nii) saves successfully!
4426
Permutation Done.
File(group/r-3D-head-centered_pat.nii) saves successfully!
9586
Permutation Done.
File(group/phi-head-centered_pat.nii) saves successfully!
29177
Permutation Done.
File(group/2D_cartesian_avg_pat.nii) saves successfully!
8268
Permutation Done.
File(group/2D_polar_avg_pat.nii) saves successfully!
16916
Permutation Done.
File(group/3D_cartesian_avg_pat.nii) saves successfully!
5316
Permutation Done.
File(group/3D_cylindrical_avg_pat.nii) saves successfully!
5388
Permutation Done.
File(group

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [24]:
subs = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
corrs = np.zeros([9, 85, 70, 67, len(subs)])
i = 0
for sub in subs:
    sub_corrs = np.load('sub'+sub+'/partialcorrs_amp_9.npy')[:, :, :, :, 0]
    corrs[:, :, :, :, i] = sub_corrs
    i += 1
avgcorrs = np.average(corrs, axis=4)
print(avgcorrs.shape)

thresholdcorrs = np.load('perm/amp_perm_threshold_9_990.npy')

titles = ['X', 'Y', 'Z', 'r', 'theta', 'r-3D', 'phi',
          'r-3D-head-centered', 'phi-head-centered']
for i in range(9):
    corr_perm_save_nii(avgcorrs[i], thresholdcorrs[i], affine, maskfile='ch2better.nii.gz',
                       filename='group/'+titles[i]+'_amp.nii',
                       size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
    
avgcorrs_2D_cartesian_avg = (avgcorrs[0] + avgcorrs[1])/2
avgcorrs_2D_polar_avg = (avgcorrs[3] + avgcorrs[4])/2
avgcorrs_3D_cartesian_avg = (avgcorrs[0] + avgcorrs[1] + avgcorrs[2])/3
avgcorrs_3D_cylindrical_avg = (avgcorrs[3] + avgcorrs[4] + avgcorrs[2])/3
avgcorrs_3D_spherical_avg = (avgcorrs[5] + avgcorrs[4] + avgcorrs[6])/3
avgcorrs_3D_spherical_hc_avg = (avgcorrs[7] + avgcorrs[4] + avgcorrs[8])/3
thresholdcorrs_2D_cartesian_avg = (thresholdcorrs[0] + thresholdcorrs[1])/2
thresholdcorrs_2D_polar_avg = (thresholdcorrs[3] + thresholdcorrs[4])/2
thresholdcorrs_3D_cartesian_avg = (thresholdcorrs[0] + thresholdcorrs[1] + thresholdcorrs[2])/3
thresholdcorrs_3D_cylindrical_avg = (thresholdcorrs[3] + thresholdcorrs[4] + thresholdcorrs[2])/3
thresholdcorrs_3D_spherical_avg = (thresholdcorrs[5] + thresholdcorrs[4] + thresholdcorrs[6])/3
thresholdcorrs_3D_spherical_hc_avg = (thresholdcorrs[7] + thresholdcorrs[4] + thresholdcorrs[8])/3
corr_perm_save_nii(avgcorrs_2D_cartesian_avg, thresholdcorrs_2D_cartesian_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_cartesian_avg_amp.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_2D_polar_avg, thresholdcorrs_2D_polar_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_polar_avg_amp.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D_cartesian_avg, thresholdcorrs_3D_cartesian_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_cartesian_avg_amp.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D_cylindrical_avg, thresholdcorrs_3D_cylindrical_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_cylindrical_avg_amp.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D_spherical_avg, thresholdcorrs_3D_spherical_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_spherical_avg_amp.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D_spherical_hc_avg, thresholdcorrs_3D_spherical_hc_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_spherical_hc_avg_amp.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

coords = ['2D_cartesian', '2D_polar',
          '3D_cartesian', '3D_cylindrical', '3D_spherical', '3D_spherical_hc']
features = ['X', 'Y', 'Z', 'r', 'theta', 'r-3D', 'phi',
            'r-3D-head-centered', 'phi-head-centered']

data6 = np.zeros([6, 87, 72, 69])
for i in range(len(coords)):
    data6[i] = nib.load('group/'+coords[i]+'_avg_amp.nii').get_fdata()

data9 = np.zeros([9, 87, 72, 69])
for i in range(len(features)):
    data9[i] = nib.load('group/'+features[i]+'_amp.nii').get_fdata()

for i in range(2):
    data_argmax = np.zeros([87, 72, 69])
    for x in range(87):
        for y in range(72):
            for z in range(69):
                if data6[i, x, y, z] > 0 and np.argmax(data6[:2, x, y, z]) == i:
                    data_argmax[x, y, z] = data6[i, x, y, z]
    file = nib.Nifti1Image(data_argmax, affine)
    nib.save(file, 'group/coord_argmax_'+coords[i]+'_amp.nii')
    
for i in range(4):
    data_argmax = np.zeros([87, 72, 69])
    for x in range(87):
        for y in range(72):
            for z in range(69):
                if data6[i+2, x, y, z] > 0 and np.argmax(data6[2:, x, y, z]) == i:
                    data_argmax[x, y, z] = data6[i+2, x, y, z]
    file = nib.Nifti1Image(data_argmax, affine)
    nib.save(file, 'group/coord_argmax_'+coords[i+2]+'_amp.nii')

data_emergent = np.zeros([6, 87, 72, 69])
data_complete = np.zeros([6, 87, 72, 69])

for x in range(87):
    for y in range(72):
        for z in range(69):
            if data6[0, x, y, z] > 0:
                if data9[0, x, y, z] > 0 or data9[1, x, y, z] > 0:
                    data_emergent[0, x, y, z] = data6[0, x, y, z]
                if data9[0, x, y, z] > 0 and data9[1, x, y, z] > 0:
                    data_complete[0, x, y, z] = data6[0, x, y, z]
            if data6[1, x, y, z] > 0:
                if data9[3, x, y, z] > 0 or data9[4, x, y, z] > 0:
                    data_emergent[1, x, y, z] = data6[1, x, y, z]
                if data9[3, x, y, z] > 0 and data9[4, x, y, z] > 0:
                    data_complete[1, x, y, z] = data6[1, x, y, z]
            if data6[2, x, y, z] > 0:
                if data9[0, x, y, z] > 0 or data9[1, x, y, z] > 0:
                    data_emergent[2, x, y, z] = data6[2, x, y, z]
                if data9[0, x, y, z] > 0 and data9[1, x, y, z] > 0 and data9[2, x, y, z] > 0:
                    data_complete[2, x, y, z] = data6[2, x, y, z]
            if data6[3, x, y, z] > 0:
                if data9[3, x, y, z] > 0:
                    data_emergent[3, x, y, z] = data6[3, x, y, z]
                if data9[3, x, y, z] > 0 and data9[4, x, y, z] > 0 and data9[2, x, y, z] > 0:
                    data_complete[3, x, y, z] = data6[3, x, y, z]
            if data6[4, x, y, z] > 0:
                if data9[5, x, y, z] > 0 or data9[6, x, y, z] > 0:
                    data_emergent[4, x, y, z] = data6[4, x, y, z]
                if data9[5, x, y, z] > 0 and data9[4, x, y, z] > 0 and data9[6, x, y, z] > 0:
                    data_complete[4, x, y, z] = data6[4, x, y, z]
            if data6[5, x, y, z] > 0:
                if data9[7, x, y, z] > 0 or data9[8, x, y, z] > 0:
                    data_emergent[5, x, y, z] = data6[5, x, y, z]
                if data9[7, x, y, z] > 0 and data9[4, x, y, z] > 0 and data9[8, x, y, z] > 0:
                    data_complete[5, x, y, z] = data6[5, x, y, z]
for i in range(6):
    file = nib.Nifti1Image(data_emergent[i], affine)
    nib.save(file, 'group/coord_emergent_'+coords[i]+'_amp.nii')
    file = nib.Nifti1Image(data_complete[i], affine)
    nib.save(file, 'group/coord_complete_'+coords[i]+'_amp.nii')

max_3d_polar = np.zeros([85, 70, 67])

stats = np.zeros([85, 70, 67])
for x in range(85):
    for y in range(70):
        for z in range(67):
            if (avgcorrs_3D_cylindrical_avg[x, y, z]>thresholdcorrs_3D_cylindrical_avg[x, y, z]) or (avgcorrs_3D_spherical_avg[x, y, z]>thresholdcorrs_3D_spherical_avg[x, y, z]) or (avgcorrs_3D_spherical_hc_avg[x, y, z]>thresholdcorrs_3D_spherical_hc_avg[x, y, z]):
                max_3d_polar[x, y, z] = np.max([avgcorrs_3D_cylindrical_avg[x, y, z], avgcorrs_3D_spherical_avg[x, y, z], avgcorrs_3D_spherical_hc_avg[x, y, z]])
                stats[x, y, z] = 1  
corr_perm_save_nii1(max_3d_polar, stats, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_polar_avg_max_amp.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

cartesian_polar_2D = avgcorrs_2D_cartesian_avg - avgcorrs_2D_polar_avg
cartesian_polar_3D = avgcorrs_3D_cartesian_avg - max_3d_polar

stats_2D = np.zeros([85, 70, 67])
stats_3D = np.zeros([85, 70, 67])
for x in range(85):
    for y in range(70):
        for z in range(67):
            if (avgcorrs_2D_cartesian_avg[x, y, z] > thresholdcorrs_2D_cartesian_avg[x, y, z]) or (avgcorrs_2D_polar_avg[x, y, z] > thresholdcorrs_2D_polar_avg[x, y, z]):
                stats_2D[x, y, z] = 1
            if (avgcorrs_3D_cartesian_avg[x, y, z] > thresholdcorrs_3D_cartesian_avg[x, y, z]) or (avgcorrs_3D_cylindrical_avg[x, y, z]>thresholdcorrs_3D_cylindrical_avg[x, y, z]) or (avgcorrs_3D_spherical_avg[x, y, z]>thresholdcorrs_3D_spherical_avg[x, y, z]) or (avgcorrs_3D_spherical_hc_avg[x, y, z]>thresholdcorrs_3D_spherical_hc_avg[x, y, z]):
                stats_3D[x, y, z] = 1
corr_perm_save_nii1(cartesian_polar_2D, stats_2D, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_cartesian_polar_amp.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii1(cartesian_polar_3D, stats_3D, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_cartesian_polar_amp.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

avgcorrs_2D = (avgcorrs[0] + avgcorrs[1] + avgcorrs[3] + avgcorrs[4])/4
avgcorrs_3D = (avgcorrs[5] + avgcorrs[6] + avgcorrs[7] + avgcorrs[8])/4
thresholdcorrs_2D = (thresholdcorrs[0] + thresholdcorrs[1] + thresholdcorrs[3] + thresholdcorrs[4])/4
thresholdcorrs_3D = (thresholdcorrs[5] + thresholdcorrs[6] + thresholdcorrs[7] + thresholdcorrs[8])/4
corr_perm_save_nii(avgcorrs_2D, thresholdcorrs_2D, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_features_avg_amp.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D, thresholdcorrs_3D, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_features_avg_amp.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

stats_2D = np.zeros([85, 70, 67])
stats_3D = np.zeros([85, 70, 67])
for x in range(85):
    for y in range(70):
        for z in range(67):
            if (avgcorrs[0, x, y, z] > thresholdcorrs[0, x, y, z]) or (avgcorrs[1, x, y, z] > thresholdcorrs[1, x, y, z])or (avgcorrs[3, x, y, z] > thresholdcorrs[3, x, y, z]) or (avgcorrs[4, x, y, z] > thresholdcorrs[4, x, y, z]):
                stats_2D[x, y, z] = 1
            if (avgcorrs[5, x, y, z] > thresholdcorrs[5, x, y, z]) or (avgcorrs[6, x, y, z] > thresholdcorrs[6, x, y, z])or (avgcorrs[7, x, y, z] > thresholdcorrs[7, x, y, z]) or (avgcorrs[8, x, y, z] > thresholdcorrs[8, x, y, z]):
                stats_3D[x, y, z] = 1
corr_perm_save_nii1(avgcorrs_2D, stats_2D, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_features_avg_and_amp.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii1(avgcorrs_3D, stats_3D, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_features_avg_and_amp.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

(9, 85, 70, 67)
19827
Permutation Done.
File(group/X_amp.nii) saves successfully!
10895
Permutation Done.
File(group/Y_amp.nii) saves successfully!
2273
Permutation Done.
File(group/Z_amp.nii) saves successfully!
10016
Permutation Done.
File(group/r_amp.nii) saves successfully!
19420
Permutation Done.
File(group/theta_amp.nii) saves successfully!
12536
Permutation Done.
File(group/r-3D_amp.nii) saves successfully!
7609
Permutation Done.
File(group/phi_amp.nii) saves successfully!
9722
Permutation Done.
File(group/r-3D-head-centered_amp.nii) saves successfully!
5604
Permutation Done.
File(group/phi-head-centered_amp.nii) saves successfully!
11747
Permutation Done.
File(group/2D_cartesian_avg_amp.nii) saves successfully!
11165
Permutation Done.
File(group/2D_polar_avg_amp.nii) saves successfully!
3289
Permutation Done.
File(group/3D_cartesian_avg_amp.nii) saves successfully!
5468
Permutation Done.
File(group/3D_cylindrical_avg_amp.nii) saves successfully!
7198
Permutation Done.
File(grou

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [62]:
subs = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

avgcorrs_comb = np.zeros([2, 9, 85, 70, 67])
thresholdcorrs_comb = np.zeros([2, 9, 85, 70, 67])

corrs = np.zeros([9, 85, 70, 67, len(subs)])
i = 0
for sub in subs:
    sub_corrs = np.load('sub'+sub+'/partialcorrs_9.npy')[:, :, :, :, 0]
    corrs[:, :, :, :, i] = sub_corrs
    i += 1
avgcorrs = np.average(corrs, axis=4)
print(avgcorrs.shape)

thresholdcorrs = np.load('perm/perm_threshold_9_990.npy')

avgcorrs_comb[0] = avgcorrs
thresholdcorrs_comb[0] = thresholdcorrs

corrs = np.zeros([9, 85, 70, 67, len(subs)])
i = 0
for sub in subs:
    sub_corrs = np.load('sub'+sub+'/partialcorrs_amp_9.npy')[:, :, :, :, 0]
    corrs[:, :, :, :, i] = sub_corrs
    i += 1
avgcorrs = np.average(corrs, axis=4)
print(avgcorrs.shape)

thresholdcorrs = np.load('perm/amp_perm_threshold_9_990.npy')

avgcorrs_comb[1] = avgcorrs
thresholdcorrs_comb[1] = thresholdcorrs

stats = np.zeros([2, 9, 85, 70, 67])
for i in range(2):
    for j in range(9):
        for x in range(85):
            for y in range(70):
                for z in range(67):
                    if avgcorrs_comb[i, j, x, y, z] > thresholdcorrs_comb[i, j, x, y, z]:
                        stats[i, j, x, y, z] = 1

for i in range(9):
    for x in range(85):
        for y in range(70):
            for z in range(67):
                if np.sum(stats[:, i, x, y, z]) == 1:
                    if stats[0, i, x, y, z] == 1:
                        avgcorrs[i, x, y, z] = avgcorrs_comb[0, i, x, y, z]
                        thresholdcorrs[i, x, y, z] = thresholdcorrs_comb[0, i, x, y, z]
                    else:
                        avgcorrs[i, x, y, z] = avgcorrs_comb[1, i, x, y, z]
                        thresholdcorrs[i, x, y, z] = thresholdcorrs_comb[1, i, x, y, z]
                else:
                    if avgcorrs_comb[0, i, x, y, z] > avgcorrs_comb[1, i, x, y, z]:
                        avgcorrs[i, x, y, z] = avgcorrs_comb[0, i, x, y, z]
                        thresholdcorrs[i, x, y, z] = thresholdcorrs_comb[0, i, x, y, z]
                    else:
                        avgcorrs[i, x, y, z] = avgcorrs_comb[1, i, x, y, z]
                        thresholdcorrs[i, x, y, z] = thresholdcorrs_comb[1, i, x, y, z]
print('done')

titles = ['X', 'Y', 'Z', 'r', 'theta', 'r-3D', 'phi',
          'r-3D-head-centered', 'phi-head-centered']
for i in range(9):
    corr_perm_save_nii(avgcorrs[i], thresholdcorrs[i], affine, maskfile='ch2better.nii.gz',
                       filename='group/'+titles[i]+'_comb.nii',
                       size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
    
avgcorrs_2D_cartesian_avg = (avgcorrs[0] + avgcorrs[1])/2
avgcorrs_2D_polar_avg = (avgcorrs[3] + avgcorrs[4])/2
avgcorrs_3D_cartesian_avg = (avgcorrs[0] + avgcorrs[1] + avgcorrs[2])/3
avgcorrs_3D_cylindrical_avg = (avgcorrs[3] + avgcorrs[4] + avgcorrs[2])/3
avgcorrs_3D_spherical_avg = (avgcorrs[5] + avgcorrs[4] + avgcorrs[6])/3
avgcorrs_3D_spherical_hc_avg = (avgcorrs[7] + avgcorrs[4] + avgcorrs[8])/3
thresholdcorrs_2D_cartesian_avg = (thresholdcorrs[0] + thresholdcorrs[1])/2
thresholdcorrs_2D_polar_avg = (thresholdcorrs[3] + thresholdcorrs[4])/2
thresholdcorrs_3D_cartesian_avg = (thresholdcorrs[0] + thresholdcorrs[1] + thresholdcorrs[2])/3
thresholdcorrs_3D_cylindrical_avg = (thresholdcorrs[3] + thresholdcorrs[4] + thresholdcorrs[2])/3
thresholdcorrs_3D_spherical_avg = (thresholdcorrs[5] + thresholdcorrs[4] + thresholdcorrs[6])/3
thresholdcorrs_3D_spherical_hc_avg = (thresholdcorrs[7] + thresholdcorrs[4] + thresholdcorrs[8])/3
corr_perm_save_nii(avgcorrs_2D_cartesian_avg, thresholdcorrs_2D_cartesian_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_cartesian_avg_comb.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_2D_polar_avg, thresholdcorrs_2D_polar_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_polar_avg_comb.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D_cartesian_avg, thresholdcorrs_3D_cartesian_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_cartesian_avg_comb.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D_cylindrical_avg, thresholdcorrs_3D_cylindrical_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_cylindrical_avg_comb.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D_spherical_avg, thresholdcorrs_3D_spherical_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_spherical_avg_comb.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D_spherical_hc_avg, thresholdcorrs_3D_spherical_hc_avg, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_spherical_hc_avg_comb.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

coords = ['2D_cartesian', '2D_polar',
          '3D_cartesian', '3D_cylindrical', '3D_spherical', '3D_spherical_hc']
features = ['X', 'Y', 'Z', 'r', 'theta', 'r-3D', 'phi',
            'r-3D-head-centered', 'phi-head-centered']

data6 = np.zeros([6, 87, 72, 69])
for i in range(len(coords)):
    data6[i] = nib.load('group/'+coords[i]+'_avg_comb.nii').get_fdata()

data9 = np.zeros([9, 87, 72, 69])
for i in range(len(features)):
    data9[i] = nib.load('group/'+features[i]+'_comb.nii').get_fdata()

for i in range(2):
    data_argmax = np.zeros([87, 72, 69])
    for x in range(87):
        for y in range(72):
            for z in range(69):
                if data6[i, x, y, z] > 0 and np.argmax(data6[:2, x, y, z]) == i:
                    data_argmax[x, y, z] = data6[i, x, y, z]
    file = nib.Nifti1Image(data_argmax, affine)
    nib.save(file, 'group/coord_argmax_'+coords[i]+'_comb.nii')
    
for i in range(4):
    data_argmax = np.zeros([87, 72, 69])
    for x in range(87):
        for y in range(72):
            for z in range(69):
                if data6[i+2, x, y, z] > 0 and np.argmax(data6[2:, x, y, z]) == i:
                    data_argmax[x, y, z] = data6[i+2, x, y, z]
    file = nib.Nifti1Image(data_argmax, affine)
    nib.save(file, 'group/coord_argmax_'+coords[i+2]+'_comb.nii')

data_emergent = np.zeros([6, 87, 72, 69])
data_complete = np.zeros([6, 87, 72, 69])

for x in range(87):
    for y in range(72):
        for z in range(69):
            if data6[0, x, y, z] > 0:
                if data9[0, x, y, z] > 0 or data9[1, x, y, z] > 0:
                    data_emergent[0, x, y, z] = data6[0, x, y, z]
                if data9[0, x, y, z] > 0 and data9[1, x, y, z] > 0:
                    data_complete[0, x, y, z] = data6[0, x, y, z]
            if data6[1, x, y, z] > 0:
                if data9[3, x, y, z] > 0 or data9[4, x, y, z] > 0:
                    data_emergent[1, x, y, z] = data6[1, x, y, z]
                if data9[3, x, y, z] > 0 and data9[4, x, y, z] > 0:
                    data_complete[1, x, y, z] = data6[1, x, y, z]
            if data6[2, x, y, z] > 0:
                if data9[0, x, y, z] > 0 or data9[1, x, y, z] > 0:
                    data_emergent[2, x, y, z] = data6[2, x, y, z]
                if data9[0, x, y, z] > 0 and data9[1, x, y, z] > 0 and data9[2, x, y, z] > 0:
                    data_complete[2, x, y, z] = data6[2, x, y, z]
            if data6[3, x, y, z] > 0:
                if data9[3, x, y, z] > 0:
                    data_emergent[3, x, y, z] = data6[3, x, y, z]
                if data9[3, x, y, z] > 0 and data9[4, x, y, z] > 0 and data9[2, x, y, z] > 0:
                    data_complete[3, x, y, z] = data6[3, x, y, z]
            if data6[4, x, y, z] > 0:
                if data9[5, x, y, z] > 0 or data9[6, x, y, z] > 0:
                    data_emergent[4, x, y, z] = data6[4, x, y, z]
                if data9[5, x, y, z] > 0 and data9[4, x, y, z] > 0 and data9[6, x, y, z] > 0:
                    data_complete[4, x, y, z] = data6[4, x, y, z]
            if data6[5, x, y, z] > 0:
                if data9[7, x, y, z] > 0 or data9[8, x, y, z] > 0:
                    data_emergent[5, x, y, z] = data6[5, x, y, z]
                if data9[7, x, y, z] > 0 and data9[4, x, y, z] > 0 and data9[8, x, y, z] > 0:
                    data_complete[5, x, y, z] = data6[5, x, y, z]
for i in range(6):
    file = nib.Nifti1Image(data_emergent[i], affine)
    nib.save(file, 'group/coord_emergent_'+coords[i]+'_comb.nii')
    file = nib.Nifti1Image(data_complete[i], affine)
    nib.save(file, 'group/coord_complete_'+coords[i]+'_comb.nii')

max_3d_polar = np.zeros([85, 70, 67])

stats = np.zeros([85, 70, 67])
for x in range(85):
    for y in range(70):
        for z in range(67):
            if (avgcorrs_3D_cylindrical_avg[x, y, z]>thresholdcorrs_3D_cylindrical_avg[x, y, z]) or (avgcorrs_3D_spherical_avg[x, y, z]>thresholdcorrs_3D_spherical_avg[x, y, z]) or (avgcorrs_3D_spherical_hc_avg[x, y, z]>thresholdcorrs_3D_spherical_hc_avg[x, y, z]):
                max_3d_polar[x, y, z] = np.max([avgcorrs_3D_cylindrical_avg[x, y, z], avgcorrs_3D_spherical_avg[x, y, z], avgcorrs_3D_spherical_hc_avg[x, y, z]])
                stats[x, y, z] = 1  
corr_perm_save_nii1(max_3d_polar, stats, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_polar_avg_max_comb.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

cartesian_polar_2D = avgcorrs_2D_cartesian_avg - avgcorrs_2D_polar_avg
cartesian_polar_3D = avgcorrs_3D_cartesian_avg - max_3d_polar

stats_2D = np.zeros([85, 70, 67])
stats_3D = np.zeros([85, 70, 67])
for x in range(85):
    for y in range(70):
        for z in range(67):
            if (avgcorrs_2D_cartesian_avg[x, y, z] > thresholdcorrs_2D_cartesian_avg[x, y, z]) or (avgcorrs_2D_polar_avg[x, y, z] > thresholdcorrs_2D_polar_avg[x, y, z]):
                stats_2D[x, y, z] = 1
            if (avgcorrs_3D_cartesian_avg[x, y, z] > thresholdcorrs_3D_cartesian_avg[x, y, z]) or (avgcorrs_3D_cylindrical_avg[x, y, z]>thresholdcorrs_3D_cylindrical_avg[x, y, z]) or (avgcorrs_3D_spherical_avg[x, y, z]>thresholdcorrs_3D_spherical_avg[x, y, z]) or (avgcorrs_3D_spherical_hc_avg[x, y, z]>thresholdcorrs_3D_spherical_hc_avg[x, y, z]):
                stats_3D[x, y, z] = 1
corr_perm_save_nii1(cartesian_polar_2D, stats_2D, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_cartesian_polar_comb.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii1(cartesian_polar_3D, stats_3D, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_cartesian_polar_comb.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

avgcorrs_2D = (avgcorrs[0] + avgcorrs[1] + avgcorrs[3] + avgcorrs[4])/4
avgcorrs_3D = (avgcorrs[5] + avgcorrs[6] + avgcorrs[7] + avgcorrs[8])/4
thresholdcorrs_2D = (thresholdcorrs[0] + thresholdcorrs[1] + thresholdcorrs[3] + thresholdcorrs[4])/4
thresholdcorrs_3D = (thresholdcorrs[5] + thresholdcorrs[6] + thresholdcorrs[7] + thresholdcorrs[8])/4
corr_perm_save_nii(avgcorrs_2D, thresholdcorrs_2D, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_features_avg_comb.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii(avgcorrs_3D, thresholdcorrs_3D, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_features_avg_comb.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

stats_2D = np.zeros([85, 70, 67])
stats_3D = np.zeros([85, 70, 67])
for x in range(85):
    for y in range(70):
        for z in range(67):
            if (avgcorrs[0, x, y, z] > thresholdcorrs[0, x, y, z]) or (avgcorrs[1, x, y, z] > thresholdcorrs[1, x, y, z])or (avgcorrs[3, x, y, z] > thresholdcorrs[3, x, y, z]) or (avgcorrs[4, x, y, z] > thresholdcorrs[4, x, y, z]):
                stats_2D[x, y, z] = 1
            if (avgcorrs[5, x, y, z] > thresholdcorrs[5, x, y, z]) or (avgcorrs[6, x, y, z] > thresholdcorrs[6, x, y, z])or (avgcorrs[7, x, y, z] > thresholdcorrs[7, x, y, z]) or (avgcorrs[8, x, y, z] > thresholdcorrs[8, x, y, z]):
                stats_3D[x, y, z] = 1
corr_perm_save_nii1(avgcorrs_2D, stats_2D, affine, maskfile='ch2better.nii.gz',
                   filename='group/2D_features_avg_and_comb.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))
corr_perm_save_nii1(avgcorrs_3D, stats_3D, affine, maskfile='ch2better.nii.gz',
                   filename='group/3D_features_avg_and_comb.nii',
                   size=[87, 72, 69], cluster_correction=False, nsubs=len(subs))

(9, 85, 70, 67)
(9, 85, 70, 67)
done
48125
Permutation Done.
File(group/X_comb.nii) saves successfully!
33496
Permutation Done.
File(group/Y_comb.nii) saves successfully!
5582
Permutation Done.
File(group/Z_comb.nii) saves successfully!
13116
Permutation Done.
File(group/r_comb.nii) saves successfully!
27032
Permutation Done.
File(group/theta_comb.nii) saves successfully!
25282
Permutation Done.
File(group/r-3D_comb.nii) saves successfully!
13770
Permutation Done.
File(group/phi_comb.nii) saves successfully!
13870
Permutation Done.
File(group/r-3D-head-centered_comb.nii) saves successfully!
14847
Permutation Done.
File(group/phi-head-centered_comb.nii) saves successfully!
37668
Permutation Done.
File(group/2D_cartesian_avg_comb.nii) saves successfully!
15271
Permutation Done.
File(group/2D_polar_avg_comb.nii) saves successfully!
20760
Permutation Done.
File(group/3D_cartesian_avg_comb.nii) saves successfully!
9739
Permutation Done.
File(group/3D_cylindrical_avg_comb.nii) saves successf

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [25]:
tal_nii = nib.load('sample_TAL_image.nii.gz')
tal_data = tal_nii.get_fdata()[:, :, :, 0]
sample_affine = tal_nii.affine

titles = ['X', 'Y', 'Z', 'r', 'theta', 'r-3D', 'phi', 'r-3D-head-centered', 'phi-head-centered',
          'coord_emergent_2D_cartesian', 'coord_emergent_2D_polar', 'coord_emergent_3D_cartesian',
          'coord_emergent_3D_cylindrical', 'coord_emergent_3D_spherical', 'coord_emergent_3D_spherical_hc',
          'coord_complete_2D_cartesian', 'coord_complete_2D_polar', 'coord_complete_3D_cartesian',
          'coord_complete_3D_cylindrical', 'coord_complete_3D_spherical', 'coord_complete_3D_spherical_hc',
          '2D_cartesian_avg', '2D_polar_avg', '3D_cartesian_avg', '3D_cylindrical_avg', '3D_spherical_avg',
          '3D_spherical_hc_avg',
          '3D_polar_avg_max',
          '2D_cartesian_polar', '3D_cartesian_polar',
          '2D_features_avg', '3D_features_avg', '2D_features_avg_and', '3D_features_avg_and',
          'X_11', 'Y_11', 'Z_11', 'r_11', 'theta_11', 'r-3D_11', 'phi_11', '2D-Integration_11', '3D-Integration_11',
          'r-3D-head-centered_11', 'phi-head-centered_11']
titles = ['2D-Integration_integration', '3D-Integration_integration']

for con in ['_pat', '_amp', '_comb']:
    for i in range(len(titles)):
        nii = nib.load('group/'+titles[i]+con+'.nii')
        data = nii.get_fdata()
        voxels = np.argwhere(data > 0)
        print(titles[i] + ' NofVoxels:' + str(voxels.shape[0]))
        coords = vox2mm(voxels, affine)
        sample_voxels = mm2vox(coords, sample_affine)
        n = sample_voxels.shape[0]
    
        for j in range(n):
            if tal_data[sample_voxels[j, 0], sample_voxels[j, 1], sample_voxels[j, 2]] == 0:
                data[voxels[j, 0], voxels[j, 1], voxels[j, 2]] = 0
        file = nib.Nifti1Image(data, affine)
    
        file = threshold_img(file, threshold=0, cluster_threshold=30)
        print(np.argwhere(file.get_fdata()).shape[0])
    
        nib.save(file, 'group/'+titles[i]+con+'.nii')

2D-Integration_integration NofVoxels:40809


/var/folders/48/ln_7q6vj41sdf4mst103k9q80000gn/T/ipykernel_63885/2547998996.py:34: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  file = threshold_img(file, threshold=0, cluster_threshold=30)


35501
3D-Integration_integration NofVoxels:4302
1061
2D-Integration_integration NofVoxels:29243
21923
3D-Integration_integration NofVoxels:15058
7046
2D-Integration_integration NofVoxels:52972
44435
3D-Integration_integration NofVoxels:18695
9648
